# Neural Network

In [ ]:
!pip install pyyaml numpy scipy matplotlib ipython jupyter pandas sympy nose opencv-python opencv-contrib-python imutils

In [1]:
import sys
sys.path.append('../src')

import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

import yaml

import matplotlib.pyplot as plt

## Load train/test data

In [2]:
hparams = yaml.safe_load(open('../src/hparams.yaml'))

hparams

{'learning_rate': 0.01, 'batch_size': 16, 'num_hidden_layers': 4}

In [3]:
import data.load_data as load_data

In [4]:
wiki_df = load_data.load_wiki_df_from_csv('../data/processed/wiki_df.csv')

wiki_df

,full_path,gender,age
0,17/10000217_1981-05-05_2009.jpg,1.0,28
1,12/100012_1948-07-03_2008.jpg,1.0,60
2,16/10002116_1971-05-31_2012.jpg,0.0,41
3,02/10002702_1960-11-09_2012.jpg,0.0,52
4,41/10003541_1937-09-27_1971.jpg,1.0,34
...,...,...,...
22573,38/9996938_1937-02-15_1968.jpg,1.0,31
22574,46/9996946_1943-11-01_1968.jpg,1.0,25
22575,49/9996949_1937-04-17_1963.jpg,1.0,26
22576,09/9998109_1972-12-27_2013.jpg,1.0,41


In [5]:
from data.WIKISequence import WIKISequence

In [6]:
wiki_generator = WIKISequence(wiki_df, 'age', 8)

print("batches:", len(wiki_generator))
print("X.shape:", wiki_generator[0][0].shape)
print("Y.age:", wiki_generator[0][1])

batches: 2823
X.shape: (8, 224, 224, 3)
Y.age: [28 60 41 52 34 42 36 23]


## Experiment A

* Ako prvé sa pokúsime vytvoriť NN podobnú VGG. Podľa [WEEK_7 lab](https://github.com/matus-pikuliak/neural_networks_at_fiit/blob/92b24eef8e6444c43a22e8fa51a349b3b1043a7c/week_7/week_7.ipynb), alebo iného tutoriálu
* Natrénujeme ju na už predspracovanom datasete
* Jej výsledok použijeme ako štartovaciu čiaru
* Túto sieť budeme rozširovať o ďalšie vrstvy a parametre
* V projekte ponecháme sieť s najlepším skóre
* Dole pripájam referenčnú ukážku siete. [Zdroj](https://www.pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/?__s)

In [7]:
import models.MiniVGGNetBuilder as MiniVGGNetBuilder

In [8]:
dir(MiniVGGNetBuilder)

['MiniVGGNetModel',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'create_new_model',
 'create_new_regression_model',
 'keras']

---
### EXPERIMENT 1: MiniVGG ako regresia pre gender class

In [10]:
vgg_regress_model = MiniVGGNetBuilder.create_new_regression_model()

In [11]:
callbacks = [
    #keras.callbacks.TensorBoard(
    #    log_dir=os.path.join("logs", timestamp()),
    #    histogram_freq=1,
    #    profile_batch=0)
]

In [12]:
BATCH_SIZE = 16

# Split 0.7 / 0.3
wiki_generator_train = WIKISequence(wiki_df[0:15052], 'gender', BATCH_SIZE)
wiki_generator_test  = WIKISequence(wiki_df[15052:22578], 'gender', BATCH_SIZE)

- Otestujeme či funguje načítavanie dát cez generator
- Skúsime fitnúť model (trainset == validset)
- Málo epoch, lebo som na CPU

In [19]:
EPOCH = 1

vgg_regress_model.fit_generator(
    wiki_generator_test,
    epochs=EPOCH,
    validation_data=wiki_generator_test
)

471/471 [==============================] - 1085s 2s/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan


---
### EXPERIMENT 2: MiniVGG ako regresia pre age

In [25]:
vgg_regress_model_age = MiniVGGNetBuilder.create_new_regression_model()

In [26]:
# Zoberem prvych n data
wiki_generator_age = WIKISequence(wiki_df[0:3000], 'age', BATCH_SIZE)

In [27]:
EPOCH = 30

vgg_regress_model_age.fit_generator(
    wiki_generator_age,
    epochs=EPOCH,
    validation_data=wiki_generator_age
)

Epoch 1/30
188/188 [==============================] - 413s 2s/step - loss: 1356.0917 - mae: 35.6192 - mse: 1488.2539 - val_loss: 839.6511 - val_mae: 25.2244 - val_mse: 841.2512
Epoch 2/30
 47/188 [======>.......................] - ETA: 4:30 - loss: 816.3082 - mae: 26.6695 - mse: 904.7232

KeyboardInterrupt: 

In [28]:
import cv2

# Load single image from disk to memmory
# resize and convert to np array
def load_img(path):
    im = cv2.imread(path)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    #im = im.reshape(1, -1)[0]
    return (np.array(im) / 255.0).astype(np.float32)


In [29]:
for i in range(25):
    file_path = '../data/raw/wiki_crop/' + wiki_df['full_path'][i]
    age = wiki_df['age'][i]

    image = np.array([load_img(file_path)])
    predictions = vgg_regress_model_age.predict([image])

    prediction = predictions[0][0]
    
    print("Actual age:", age, "Predicted age:", prediction, "Diff:", age-prediction)

Actual age: 28 Predicted age: 18.966011 Diff: 9.033988952636719
Actual age: 60 Predicted age: 20.075502 Diff: 39.92449760437012
Actual age: 41 Predicted age: 12.990432 Diff: 28.009568214416504
Actual age: 52 Predicted age: 17.281443 Diff: 34.718557357788086
Actual age: 34 Predicted age: 10.211479 Diff: 23.78852081298828
Actual age: 42 Predicted age: 9.802255 Diff: 32.19774532318115
Actual age: 36 Predicted age: 18.969412 Diff: 17.030588150024414
Actual age: 23 Predicted age: 7.632596 Diff: 15.367403984069824
Actual age: 25 Predicted age: 20.725386 Diff: 4.274614334106445
Actual age: 37 Predicted age: 15.981272 Diff: 21.018728256225586
Actual age: 78 Predicted age: 16.827593 Diff: 61.172407150268555
Actual age: 42 Predicted age: 16.331415 Diff: 25.6685848236084
Actual age: 33 Predicted age: 10.487508 Diff: 22.512492179870605
Actual age: 28 Predicted age: 21.888805 Diff: 6.111194610595703
Actual age: 60 Predicted age: 21.40146 Diff: 38.59853935241699
Actual age: 13 Predicted age: 12.0935

In [19]:
batch = wiki_generator_age[0]

vgg_regress_model_age.predict(batch[0][0:1])

array([[0.442695]], dtype=float32)

In [21]:
batch[0][0:1].shape

(1, 224, 224, 3)

### EXPERIMENT 3/4: MiniVGG kategoria pre pohlavie

In [22]:
vgg_cat_model_gender = MiniVGGNetBuilder.create_new_model(2)

In [23]:
BATCH_SIZE = 16
# Zoberem prvych n data
wiki_generator_gender = WIKISequence(wiki_df[0:1000], 'gender', BATCH_SIZE)

In [24]:
EPOCH = 1

vgg_cat_model_gender.fit_generator(
    wiki_generator_gender,
    epochs=EPOCH,
    # validation_data=wiki_generator_gender
)

 3/63 [>.............................] - ETA: 1:51 - loss: 2.6059 - accuracy: 0.4583

InvalidArgumentError: Received a label value of -9223372036854775808 which is outside the valid range of [0, 2).  Label values: 1 -9223372036854775808 0 0 1 1 1 0 0 0 1 -9223372036854775808 1 1 1 0 [Op:SparseSoftmaxCrossEntropyWithLogits]

### EXPERIMENT 4/4: MiniVGG kategoria pre vek

In [9]:
vgg_cat_model_age = MiniVGGNetBuilder.create_new_model(101)

In [10]:
BATCH_SIZE = 16
# Zoberem prvych n data
wiki_generator_age = WIKISequence(wiki_df[0:3000], 'age', BATCH_SIZE)

In [11]:
EPOCH = 1

vgg_cat_model_age.fit_generator(
    wiki_generator_age,
    epochs=EPOCH,
    validation_data=wiki_generator_age
)

188/188 [==============================] - 381s 2s/step - loss: 5.6091 - accuracy: 0.0248 - val_loss: 5.0629 - val_accuracy: 0.0353


In [21]:
for i in range(25):
    file_path = '../data/raw/wiki_crop/' + wiki_df['full_path'][i]
    age = wiki_df['age'][i]

    image = np.array([load_img(file_path)])
    predictions = vgg_cat_model_age.predict([image])

    output_indexes = np.array([i for i in range(0, 101)])
    apparent_predictions = np.sum(predictions * output_indexes, axis = 1)
    
    prediction = apparent_predictions[0]
    
    print("Actual age:", age, "Predicted age:", prediction, "Diff:", age-prediction, "argmax:", np.argmax(predictions[0]))

Actual age: 28 Predicted age: 51.140328982204664 Diff: -23.140328982204664 argmax: 80
Actual age: 60 Predicted age: 52.27139389177319 Diff: 7.728606108226813 argmax: 22
Actual age: 41 Predicted age: 54.024378877395066 Diff: -13.024378877395066 argmax: 49
Actual age: 52 Predicted age: 51.864489524799865 Diff: 0.13551047520013526 argmax: 22
Actual age: 34 Predicted age: 51.405801227432676 Diff: -17.405801227432676 argmax: 22
Actual age: 42 Predicted age: 54.663264062051894 Diff: -12.663264062051894 argmax: 49
Actual age: 36 Predicted age: 53.2028395057132 Diff: -17.2028395057132 argmax: 35
Actual age: 23 Predicted age: 49.90469061565818 Diff: -26.904690615658183 argmax: 22
Actual age: 25 Predicted age: 52.711947747680824 Diff: -27.711947747680824 argmax: 22
Actual age: 37 Predicted age: 50.58010940166423 Diff: -13.580109401664231 argmax: 22
Actual age: 78 Predicted age: 52.48106994279078 Diff: 25.518930057209218 argmax: 22
Actual age: 42 Predicted age: 52.040824557218 Diff: -10.040824557

---

In [48]:
# Toto bude zle!

x = np.array([ load_img(wiki_df['full_path'][0]) ])
y = model.predict(x)

np.argmax(y)

# ===============================================

# Spravne by sa to malo robit takto. pise sa to v tom clanku od tvorcov tohto data setu

predictions = vgg_cat_model_age.predict(test_images)
 
classes = np.array([i for i in range(0, 101)])
apparent_age = np.sum(predictions * classes, axis = 1)

6

In [14]:
#Save model

tf.keras.models.save_model(
    model,
    filepath = "../models/mini-vgg-1.tf",
    overwrite=True,
    include_optimizer=True,
    save_format="tf"
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../models/mini-vgg-1.tf/assets


# Experiment 2
Cely VGG model.

Zdroj: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(256, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Convolution2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2), strides=(2,2)))
 
model.add(keras.layers.Convolution2D(4096, (7, 7), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(4096, (1, 1), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Convolution2D(2622, (1, 1)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Activation('softmax'))

In [7]:
model.load_weights('../data/vgg_face_weights.h5')

In [8]:
vgg_face_descriptor = keras.Model(inputs=model.layers[0].input, outputs=model.layers[-1].output)

Convert image as array using opencv

In [9]:
classes = 101 #0 to 100
target = wiki_df['age'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []
 
for i in range(0, wiki_df.shape[0]):
    features.append(wiki_df['img_array'].values[i])
 
features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

In [12]:
!pip install scikit-learn

     |████████████████████████████████| 6.7MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 296kB 28.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

In [ ]:
for layer in model.layers[:-7]:
    layer.trainable = False

In [15]:
base_model_output = keras.models.Sequential()
base_model_output = keras.layers.Convolution2D(101, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = keras.layers.Flatten()(base_model_output)
base_model_output = keras.layers.Activation('softmax')(base_model_output)

In [16]:
age_model = keras.Model(inputs=model.input, outputs=base_model_output)

In [17]:
age_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
scores = []
epochs = 250; batch_size = 50
 
for i in range(epochs):
    print("epoch ",i)
 
    ix_train = np.random.choice(train_x.shape[0], size=batch_size)
 
    score = age_model.fit(train_x[ix_train], train_y[ix_train], epochs=1, validation_data=(test_x, test_y))
 
    scores.append(score)

epoch  0


### Playground
---

In [108]:
# Tento code snippet uz nepotrebujeme, protoze pouzijeme 'sparse_categorical_...'

# Generate target classes

classes = 101 #0 to 100
target = wiki_df['age'].values           ### train/valid Y
target_classes = keras.utils.to_categorical(target, classes)

file_paths = wiki_df['full_path'].values ### train/valid X

N_SAMPLES = len(target)

assert(len(target) == len(file_paths))

len(target_classes)

22578

In [10]:
# from: https://stackoverflow.com/a/55666861
def change_to_right(wrong_labels):
    right_labels=[]
    for x in wrong_labels:
        for i in range(0,len(wrong_labels[0])):
            if x[i]==1:
                right_labels.append(i)
    return right_labels

#train_labels = tf.convert_to_tensor(np.array(change_to_right(train_labels)))
#test_labels = tf.convert_to_tensor(np.array(change_to_right(test_labels)))